# Experimenting With Different Replenishment Policies

This notebook demonstrates the use of different replenishment policies for nodes in supply chains. It also compares the results with AnyLogistix by implementing the same supply chain network in AnyLogistix and running simulations.

In [1]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import SupplyNetPy.Components as scm

def print_node_wise_performance(nodes_object_list):
    if not nodes_object_list:
        print("No nodes provided.")
        return

    # Pre-fetch statistics from all nodes
    stats_per_node = {node.name: node.stats.get_statistics() for node in nodes_object_list}
    stat_keys = sorted(next(iter(stats_per_node.values())).keys())

    # Determine column widths
    col_width = 25
    header = "Performance Metric".ljust(col_width)
    for name in stats_per_node:
        header += name.ljust(col_width)
    print(header)

    # Print row-wise stats
    for key in stat_keys:
        row = key.ljust(col_width)
        for name in stats_per_node:
            value = stats_per_node[name].get(key, "N/A")
            row += str(value).ljust(col_width)
        print(row)

def print_sc_performance(scnet):
    performance_keys = sorted([
        "available_inv", "avg_available_inv", "inventory_carry_cost", "inventory_spend_cost",
        "transportation_cost", "revenue", "total_cost", "profit", "demand_by_customers",
        "fulfillment_received_by_customers", "demand_by_site", "fulfillment_received_by_site",
        "total_demand", "total_fulfillment_received", "shortage", "backorders",
        "avg_cost_per_order", "avg_cost_per_item"
    ])

    print("\n--- Supply Chain Performance Summary ---\n")
    max_key_length = max(len(key) for key in performance_keys) + 2
    for key in performance_keys:
        value = scnet.get(key, "N/A")
        print(f"{key.ljust(max_key_length)}: {value}")

In [4]:
"""
Testing replenishment policies with a simple supply chain network
"""
import simpy
env = simpy.Environment()

supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)

distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', 
                                capacity=float('inf'), initial_level=0, inventory_holding_cost=0.1, 
                                replenishment_policy=scm.SSReplenishment, logging=True,
                                policy_param={'s':5,'S':20}, product_sell_price=100, product_buy_price=90)

link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 2)

demand1 = scm.Demand(env=env, ID='d1', name='Demand1', order_arrival_model=lambda: 1, order_quantity_model=lambda:10,
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)#, tolerance=float('inf'))

supplynet = scm.create_sc_net(env=env,nodes=[supplier,distributor],links=[link1],demands=[demand1])
supplynet = scm.simulate_sc_net(supplynet, sim_time=20, logging=False)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
#plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
#plt.grid()
print_node_wise_performance([distributor, demand1])
print_sc_performance(supplynet)

Performance Metric       Distributor1             Demand1                  
backorder                [0, 0]                   [0, 0]                   
demand_fulfilled         [11, 110]                [0, 0]                   
demand_placed            [7, 140]                 [20, 200]                
demand_received          [12, 120]                [0, 0]                   
fulfillment_received     [6, 120]                 [11, 110]                
inventory_carry_cost     6.0                      0                        
inventory_level          0                        0                        
inventory_spend_cost     10800                    0                        
inventory_waste          0                        0                        
node_cost                10876.0                  120                      
orders_shortage          [8, 80]                  [0, 0]                   
profit                   124.0                    -120                     
revenue     

**How do we set up the experiment?**

We set up a small single-echelon supply chain with a deterministic constant demand (e.g., 10 units per day).
The network is configured by setting values for the replenishment policy parameters (e.g., s and S in a min–max policy). The same configuration is used for implementation in both AnyLogistix and SupplyNetPy.

We then run simulations. Since there is no stochastic element in these scenarios, we expect to obtain identical results. In some cases, the results match exactly; however, in others, slight differences occur. The reason lies in how the underlying discrete-event simulation framework handles simultaneous events, specifically, the order in which events are executed. This is demonstrated in the small code snippet below.

**Scenario 1:** Suppose that at time t, both inventory replenishment and demand arrival occur.

- If the inventory is 0 and demand is processed before replenishment, the customer may leave empty-handed, depending on the backorder policy.

- If demand is processed after replenishment, the order may be fulfilled.

In [3]:
"""
This code snippet demonstrates the floating-point issue in discrete-event simulations.
It occurs due to how floats are stored and incremented. In the following example, event A is created before event B. 
Event A occurs every 0.4 units of time, and event B occurs every 0.3 units. 
At time 1.2, event A should be executed before event B. However, because of floating-point precision issues, 
event B is executed before event A.
"""

import simpy

def test_process(env):
    while True:
        print(f"eve A: {env.now}")
        yield env.timeout(0.4)

def test_process2(env):
    while True:
        print(f"eve B: {env.now}")
        yield env.timeout(0.3)

env = simpy.Environment()
env.process(test_process(env))
env.process(test_process2(env))
env.run(until=2.400001)
# This might be because of how Python stores floats and handles precision.

eve A: 0
eve B: 0
eve B: 0.3
eve A: 0.4
eve B: 0.6
eve A: 0.8
eve B: 0.8999999999999999
eve B: 1.2
eve A: 1.2000000000000002
eve B: 1.5
eve A: 1.6
eve B: 1.8
eve A: 2.0
eve B: 2.1
eve A: 2.4
eve B: 2.4


**Conflict Resolution for Simulteneous Events**

The following code snippet is to check the case of simultaneous events, and how these events are carried out by SimPy and AnyLogistix

In [ ]:
"""
Testing replenishment policies with a simple supply chain network
"""
import simpy
env = simpy.Environment()

supplier = scm.Supplier(env=env, ID='S1', name='Supplier 1', node_type='infinite_supplier',logging=False)

distributor = scm.InventoryNode(env=env, ID='D1', name='Distributor1', node_type='distributor', 
                                capacity=float('inf'), initial_level=5, inventory_holding_cost=0.1, 
                                replenishment_policy=scm.PeriodicReplenishment, logging=True,
                                policy_param={'T':1,'Q':3}, product_sell_price=100, product_buy_price=90)

link1  = scm.Link(env=env, ID='L1', source=supplier, sink=distributor, cost=10, lead_time=lambda: 2)

demand1 = scm.Demand(env=env, ID='d1', name='Demand1', order_arrival_model=lambda: 0.3, order_quantity_model=lambda:1,
                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

#demand2 = scm.Demand(env=env, ID='d2', name='Demand2', order_arrival_model=lambda: 0.4, order_quantity_model=lambda:1, 
#                     logging=True,demand_node=distributor, delivery_cost=lambda: 10, lead_time=lambda: 2)

supplynet = scm.create_sc_net(env=env,nodes=[supplier,distributor],links=[link1],demands=[demand1])
supplynet = scm.simulate_sc_net(supplynet, sim_time=10, logging=True)
inv_levels = np.array(distributor.inventory.instantaneous_levels)
#plt.plot(inv_levels[:,0], inv_levels[:,1], label='Inventory Level at D1', marker='.', color='blue')
#plt.grid()
print_node_wise_performance([distributor, demand1])

# carry cost mismatch with AnyLogistix
# till t=9, we get exactly matching results (carry cost = 1.32)
# Two simultaneous events at t=9.0 (replenishment and demand)
# For us (SupplyNetPy) demand at 9.00 is fulfilled, but in AnyLogistix demand at 9.0 is not fulfilled, however 9.9 is fulfilled
# 
# Checking simulation logs: simultaneous events at t=3.0, 6.0, 9.0
# At t=3.00, t=6.00, the demand d1 arrives first (and inv not sufficient), then inventory is replenished.
# But at 9.00 inventory is replenished first and then demand d1 comes.

INFO D1 - 0.0000:D1: Inventory levels:5, on hand:5
INFO D1 - 0.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 3 units.
INFO D1 - 0.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 0.0000:d1:Customer1:Order quantity:1, available.
INFO d1 - 0.3000:d1:Customer2:Order quantity:1, available.
INFO d1 - 0.6000:d1:Customer3:Order quantity:1, available.
INFO d1 - 0.9000:d1:Customer4:Order quantity:1, available.
INFO D1 - 1.0000:D1: Inventory levels:1, on hand:4
INFO D1 - 1.0000:D1:Replenishing inventory from supplier:Supplier 1, order placed for 3 units.
INFO D1 - 1.0000:D1:shipment in transit from supplier:Supplier 1.
INFO d1 - 1.2000:d1:Customer5:Order quantity:1, available.
INFO d1 - 1.5000:d1:Customer6: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO d1 - 1.8000:d1:Customer7: Order quantity:1 not available, inventory level:0. No tolerance! Shortage:1.
INFO D1 - 2.0000:D1:Inventory replenished. reorder_quantity=3, In

Performance Metric       Distributor1             Demand1                  
backorder                [0, 0]                   [0, 0]                   
demand_fulfilled         [23, 23]                 [0, 0]                   
demand_placed            [10, 30]                 [34, 34]                 
demand_received          [29, 29]                 [0, 0]                   
fulfillment_received     [8, 24]                  [23, 23]                 
inventory_carry_cost     1.4099999999999975       0                        
inventory_level          0                        0                        
inventory_spend_cost     2160                     0                        
inventory_waste          0                        0                        
node_cost                2261.41                  290                      
orders_shortage          [5, 5]                   [0, 0]                   
profit                   38.590000000000146       -290                     
revenue     

In [5]:
"""
We experiment with SimPy to understand how subsequent events (spawned later in time)
are ordered and executed when they are scheduled to occur at the exact same time.
"""

env = simpy.Environment()

inventory2 = simpy.Container(env, capacity=float('inf'), init=float('inf'))
inventory = simpy.Container(env, capacity=float('inf'), init=1)
inv_drop = env.event()

def demand(env, inventory):
    """
    Demand of 1 unit every 2 days, starts at day 0
    Consume if available, if not available, leave immidiately.
    """
    print(f"Process B: Demand is created.")
    global inv_drop
    while True:
        #yield env.timeout(0.0)
        print(f"{env.now}:B: Demand for 1 ")
        if inventory.level >= 1:
            yield inventory.get(1.0)
            inv_drop.succeed()
            inv_drop = env.event()
            print(f"{env.now}:B: Fulfilled, Inventory: {inventory.level}")
        else:
            print(f"{env.now}:B: not fulfilled at, Inventory: {inventory.level}")
        yield env.timeout(2.0)

def replenish(env, inventory):
    """
    Instanteneous replenishment
    Reorder quantity is always 1 unit, lead time is 2 
    Order when inventory level < 1
    (R,Q) policy with R=0, Q=1
    """
    print(f"Process A: Replenishment is created.")
    global inv_drop
    while True:
        replenished_amount = 1  # Fixed replenishment amount
        if(inventory.level < 1):
            print(f"{env.now}:A: Ordering replenishment for {replenished_amount} units")
            #yield inventory2.get(replenished_amount) # get it from the supplier inventory2
            yield env.timeout(2.0)  # Replenishment every 2 time units
            inventory.put(replenished_amount)
            print(f"{env.now}:A: Shipment received. Inventory replenished to {inventory.level}")
        else:
            yield inv_drop

replenish_event = env.process(replenish(env, inventory)) 
demand_event = env.process(demand(env, inventory))

env.run(until=10)
print(f"Inventory lvl: {inventory.level}")

# When the 'replenish' process is created before 'demand', the order is preserved for subsequent events.
# As a result, inventory is replenished before demand is processed. (This behavior is also observed in AnyLogistix.)

# Example 1:
# 0: replenishment order placed
# 0: demand arrives
# 2: replenishment received
# 2: demand arrives

# Example 2:
# 0: demand arrives (since sufficient inventory is available, no replenishment order is placed, so demand is processed first)
# 0: replenishment order placed (after demand is satisfied, inventory level drops, triggering a replenishment)
# 2: demand arrives
# 2: replenishment received

# When 'demand' is created before 'replenish', demand is processed first and then inventory is replenished,
# which may lead to unsatisfied demand (this is not the correct order).

# Another scenario: when 'yield inventory2.get()' is used, the 'replenish' process is delayed relative to 'demand',
# leading to unsatisfied demand.

# Note: According to the SimPy documentation, when two events are scheduled at the same time,
# their execution order is determined by the event_id. Each event is assigned an ID when it is created,
# so the event created earlier will be processed first.

Process A: Replenishment is created.
Process B: Demand is created.
0:B: Demand for 1 
0:B: Fulfilled, Inventory: 0.0
0:A: Ordering replenishment for 1 units
2.0:B: Demand for 1 
2.0:B: not fulfilled at, Inventory: 0.0
2.0:A: Shipment received. Inventory replenished to 1.0
4.0:B: Demand for 1 
4.0:B: Fulfilled, Inventory: 0.0
4.0:A: Ordering replenishment for 1 units
6.0:B: Demand for 1 
6.0:B: not fulfilled at, Inventory: 0.0
6.0:A: Shipment received. Inventory replenished to 1.0
8.0:B: Demand for 1 
8.0:B: Fulfilled, Inventory: 0.0
8.0:A: Ordering replenishment for 1 units
Inventory lvl: 0.0
